In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
import seaborn as sns
import more_itertools
from statsmodels.nonparametric.smoothers_lowess import lowess
import pysam
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.colors
import sklearn.cluster
from scipy.cluster.hierarchy import leaves_list
from scipy.stats import linregress
from scipy.cluster.hierarchy import fcluster, linkage
from singlecellmultiomics.utils import createRowColorDataFrame

In [ ]:
# Define chromsome order:
def sort_chromosome_names(l):
    chrom_values = []
    for chrom in l:
        chrom_value = None
        chrom = chrom.replace('chr','')
        if chrom == 'X':
            chrom_value = 99
        elif chrom == 'M':
            chrom_value = 101
        elif chrom == 'EBV':
            chrom_value = 102
        else:
            chrom_value = int(chrom)
        chrom_values.append(chrom_value)
    
    indices = sorted(range(len(chrom_values)),key=lambda x:chrom_values[x])
    return [l[idx] for idx in indices]

In [ ]:
bin_size = 500000
MAXCP = 4

In [ ]:
df = pd.concat( 
    (
    pd.read_pickle(plate) 
    for plate in list(glob(f'/home/buysdb/repos/ColonOrganoidDynamics/data/APKS*-P*/tagged.bam.table.bs_{bin_size}.CNV_reads.pickle.gz'))
    ),
    sort=True,
    axis=1
)

In [ ]:
df = df.T

In [ ]:
df = df.drop([ b for b in df.columns if b[1] in ['chrY','chrM','chrEBV'] or 'decoy' in b[1] or 'chrUn' in b[1] or b[1].endswith('_alt')  or b[1].endswith('_random')],1)

In [ ]:
def plot_loghist(x, bins):
    hist, bins = np.histogram(x, bins=bins)
    logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))
    plt.hist(x, bins=logbins)
    plt.xscale('log')

In [ ]:
med = df.sum(level=[1,2], axis=1).median(1)
total=df.sum(1)

In [ ]:

slope, intercept, r_value, p_value, std_err = linregress(med, total)

fig, ax = plt.subplots(figsize=(8,8))
plt.scatter(med,total)
x=np.linspace(0,max(med),4)
plt.plot(x, intercept + slope*x, 'r', label='fitted line')
plt.xlabel('median reads')
plt.ylabel('total reads')

In [ ]:
# Determine mappability per bin:
df_wo_alleles = df.sum(level=[1,2], axis=1)
bin_names = set(df_wo_alleles.columns)
import collections
bininfo = collections.defaultdict(collections.Counter) # bin
with open('/media/buysdb/eightA/references/hg38/simulated_CATG_single_69.mapability.stats.tsv') as f:
    for line in f:
        if len(line.strip())==0:
            continue
        chrom, pos, strand, correct, lost, gained = line.strip().split()
        bin_idx = int(int(pos)/bin_size)
        if not (chrom, bin_idx) in bin_names:
            continue
        
        bininfo[(chrom, bin_idx)]['lost'] += int(lost)
        bininfo[(chrom, bin_idx)]['gained'] += int(gained)
        bininfo[(chrom, bin_idx)]['correct'] += int(correct)
        
bininfo = pd.DataFrame(bininfo).fillna(0).T
plt.scatter( bininfo['lost'], bininfo['correct'],s=2,alpha=0.2)        

In [ ]:
# Select bins with proper mappability
sbins = bininfo[(bininfo['correct']>3000)].index

In [ ]:
# Sum allele data for all columns: (creates a non-allele specific matrix)

total = df_wo_alleles.sum(1)

df_wo_alleles_pre_normed = (df_wo_alleles[sbins]/bininfo['correct'][sbins])*bininfo['correct'][sbins].mean()

df_wo_alleles_normed = ((df_wo_alleles_pre_normed.T.fillna(0) / df_wo_alleles_pre_normed[sbins].T.mean().fillna(0)).T) * 2

df_wo_alleles_normed=df_wo_alleles_normed[df_wo_alleles_pre_normed.median(1)>0]

df_wo_alleles_normed.loc[:,'total'] = total
fig, ax = plt.subplots(figsize=(10,100))

sns.heatmap(df_wo_alleles_normed.sort_index(1).sort_values('total')['chr9'],ax=ax,vmax=4)
df_wo_alleles_normed=df_wo_alleles_normed.drop('total',1)

In [ ]:
#reference = pysam.FastaFile('/media/sf_data/references/GATK-Bundle/hg38/Homo_sapiens_assembly38.fasta')
reference = pysam.FastaFile('/media/buysdb/eightA/references/hg38/Homo_sapiens_assembly38.fasta')
chrom_sizes= dict( zip(reference.references, reference.lengths))

In [ ]:
# Extract GC percentage from reference for the selected bin size:
bins_to_gc = {}

for contig,bin_index in df_wo_alleles_normed.columns:
    
    if not (contig,bin_index) in bins_to_gc:
        sequence = reference.fetch(contig, bin_index*bin_size, (1+bin_index)*bin_size ).upper()
        gc = sequence.count('G')+sequence.count('C')
        gcrat = (gc) / ((sequence.count('A')+sequence.count('T')+gc))
        bins_to_gc[ (contig,bin_index) ] = gcrat


In [ ]:
gc_matched = df_wo_alleles_normed.T.join( pd.DataFrame({'gc':bins_to_gc}), how='left')['gc']
gc_matched

In [ ]:
# This is the global GC bias:
fig, ax = plt.subplots()
plt.scatter( gc_matched, df_wo_alleles_normed.mean() )
correction = lowess(df_wo_alleles_normed.mean(), gc_matched)
plt.plot(correction[:,0], correction[:,1], c='r')

In [ ]:
def gc_correct(args):
    observations, gc_vector,MAXCP = args
    correction = lowess(observations, gc_vector)
    return np.clip(observations / np.interp( gc_vector, correction[:,0], correction[:,1] ) , 0,MAXCP)

In [ ]:
from multiprocessing import Pool
workers = Pool(32)


In [ ]:
keep_bins=df_wo_alleles_normed.columns
gc_vector = gc_matched[keep_bins]

corrected_cells = list( workers.imap(
    gc_correct, [(row,gc_vector,MAXCP) for cell,row in df_wo_alleles_normed[keep_bins].iterrows()] ))

In [ ]:
chrom_order = sort_chromosome_names(list(set([chrom for chrom,_ in corrected_cells[0].index])))

In [ ]:
cumlen = {}
prev= 0
for chrom in chrom_order:
    cumlen[chrom]=prev
    prev+=chrom_sizes[chrom]

In [ ]:
corrected_cells = pd.concat(corrected_cells,axis=1).T.sort_index(1)[chrom_order]

In [ ]:
sns.heatmap( corrected_cells.iloc[:600,:] )

In [ ]:
fig, ax = plt.subplots()
plt.hist( df[('allele1','chr18')].sum(1) / ( df[('allele2','chr18')].sum(1) + df[('allele1','chr18')].sum(1) ), bins=50 )
pass

In [ ]:
df_wo_alleles_gc_corr = corrected_cells*2

In [ ]:
segment_bounds =  {
            'chr1': { 15, 155, 157, 201, 436},
             'chr2': {0, 135, 467},
             'chr3': {0, 252, 387},
             'chr4': {0, 372},
             'chr5': {0, 348},
             'chr6': {0, 55, 66, 68, 333},
             'chr7': {0, 302},
             'chr8': {0, 5, 8, 82, 123, 279},
             'chr9': {0, 7, 8, 69, 76, 216},
             'chr10': {0, 91, 255},
             'chr11': {0, 256},
             'chr12': {0, 142, 181, 258},
             'chr13': {0, 190},
             'chr14': {0, 173},
             'chr15': {0, 10, 16, 153},
             'chr16': {0, 150},
             'chr17': {0, 97, 152},
             'chr18': {0, 29, 32, 37, 42, 147},
             'chr19': {0, 107},
             'chr20': {0, 116},
             'chr21': {0, 63},
             'chr22': {0, 64},
             'chrX': {0, 292}}

In [ ]:
list( more_itertools.windowed(sorted(list(segment_bounds['chr1'])),2) )

In [ ]:
for chrom, bounds in segment_bounds.items():
    for region in more_itertools.windowed()

In [ ]:
finalcalls = pd.read_pickle('../../Analysis (another copy)/cnv_normaliser/figures/resources/hybrid_cn_matrix_reduced_segments_variance.pickle.gz')

In [ ]:

for bin_chrom, bin_start in selected_data.index:
    if bin_chrom!=chrom:
        continue
        
    print(bin_start*bin_size, bin_chrom==chrom, bin_start*bin_size*1000>=start, (bin_start*(1000*bin_size+1))<=end )

In [ ]:
selected_data = df_wo_alleles_gc_corr[chrom_order].loc[cell,:]
for chrom, allele, (start, end) in finalcalls.columns:
    start = start/bin_size
    end = end/bin_size
    bins = [
        (bin_chrom==chrom and bin_start*bin_size>=start and (bin_start*(bin_size+1))<=end  ) for bin_chrom, bin_start in selected_data.index
    ]
    

In [ ]:
var = df_wo_alleles_gc_corr['chr1'].diff(periods=1,axis=1).var(1)
plt.hist( var, bins=100)

var_threshold=0.6

plt.show()
df_wo_alleles_gc_corr.loc[:,'var'] =  var
df_wo_alleles_gc_corr = df_wo_alleles_gc_corr.sort_values('var').drop('var',1)


In [ ]:
show_cells = 40

ht = 2.5
lt = 1.5

fig, axes = plt.subplots(show_cells,1,figsize=(12,1*show_cells), sharex=True, sharey=True)
sns.despine()
for i,ax,cell in zip(range(show_cells), axes, [idx for idx in df_wo_alleles_gc_corr.index if idx in finalcalls.index]):
    
    
    inc = df_wo_alleles_gc_corr[chrom_order].loc[cell,:]>ht
    inc[~inc]=np.nan
    (df_wo_alleles_gc_corr[chrom_order].loc[cell,:]*inc).plot(style='.',ms=0.5,ax=ax,color=tuple(np.array((131,40,30))/255))
    
    inc = df_wo_alleles_gc_corr[chrom_order].loc[cell,:]<lt
    inc[~inc]=np.nan
    (df_wo_alleles_gc_corr[chrom_order].loc[cell,:]*inc).plot(style='.',ms=0.5,ax=ax,color=tuple(np.array((94,86,157,255))/255))
    
    inc = (df_wo_alleles_gc_corr[chrom_order].loc[cell,:]>=lt) & (df_wo_alleles_gc_corr[chrom_order].loc[cell,:]<=ht)
    inc[~inc]=np.nan
    (df_wo_alleles_gc_corr[chrom_order].loc[cell,:]*inc).plot(style='.',ms=1,ax=ax,color='grey', alpha=0.5)

    
    ax.set_ylim(-1,5)
    

    
    
    if False:
        selected_data = df_wo_alleles_gc_corr[chrom_order].loc[cell,:]
        for k in finalcalls.columns:
            chrom, allele, (start, end) = k
            if allele != 'none':
                continue
            start = start/bin_size
            end = end/bin_size
            bins = [
                1 if (bin_chrom==chrom and bin_start*bin_size>=start and (bin_start*(bin_size+1))<=end  ) else np.nan for bin_chrom, bin_start in selected_data.index
            ]

            call = finalcalls.loc[cell][k]
            if call>2:
                (selected_data * bins).plot(style='.',ms=1,ax=ax,color=(131,40,30))
            elif call==1:
                (selected_data * bins).plot(style='.',ms=1,ax=ax,color=(94,86,157))


    
    prev=None
    for binidx, (chrom, binpos) in enumerate(df_wo_alleles_gc_corr[chrom_order].columns):
        if prev is not None and chrom!=prev:
            ax.axvline(binidx-1,c='k',lw=1, )
        prev = chrom
    ax.set_yticks([0,2,4])
    #ax.set_ylabel('Copy number')
plt.savefig('raw_copy_numbers.svg')
    
    

In [ ]:
mkdir single_cell_traces

In [ ]:
show_cells = 40


ht = 2.5
lt = 1.5


for i,cell in zip(range(show_cells), [idx for idx in df_wo_alleles_gc_corr.index if idx in finalcalls.index]):
    
    fig, ax = plt.subplots(1,1,figsize=(12,1.5))
    sns.despine()
    
    inc = df_wo_alleles_gc_corr[chrom_order].loc[cell,:]>ht
    inc[~inc]=np.nan
    (df_wo_alleles_gc_corr[chrom_order].loc[cell,:]*inc).plot(style='.',ms=0.5,ax=ax,color=tuple(np.array((131,40,30))/255))
    
    inc = df_wo_alleles_gc_corr[chrom_order].loc[cell,:]<lt
    inc[~inc]=np.nan
    (df_wo_alleles_gc_corr[chrom_order].loc[cell,:]*inc).plot(style='.',ms=0.5,ax=ax,color=tuple(np.array((94,86,157,255))/255))
    
    inc = (df_wo_alleles_gc_corr[chrom_order].loc[cell,:]>=lt) & (df_wo_alleles_gc_corr[chrom_order].loc[cell,:]<=ht)
    inc[~inc]=np.nan
    (df_wo_alleles_gc_corr[chrom_order].loc[cell,:]*inc).plot(style='.',ms=1,ax=ax,color='grey', alpha=0.5)

    
    ax.set_ylim(-1,5)
    

    
  
    prev=None
    for binidx, (chrom, binpos) in enumerate(df_wo_alleles_gc_corr[chrom_order].columns):
        if prev is not None and chrom!=prev:
            ax.axvline(binidx-1,c='k',lw=1, )
        prev = chrom
    ax.set_yticks([0,2,4])
    #ax.set_ylabel('Copy number')
    plt.savefig(f'./single_cell_traces/{cell}.svg')
    plt.close()
    
    

In [ ]:
pwd

In [ ]:
show_cells = 40

fig, axes = plt.subplots(10,1,figsize=(12,10), sharex=True, sharey=True)
for i,ax in zip(range(10), axes):
    
    for chrom in chrom_order:

        #df_wo_alleles_gc_corr[chrom].iloc[i,:].plot(style='.',ms=1,ax=ax,color='grey')
        ax.set_ylim(-1,5)
        
        x_coords = df_wo_alleles_gc_corr[chrom].iloc[i,:].index.values*bin_size + cumlen[chrom]
        ax.scatter(x_coords,df_wo_alleles_gc_corr[chrom].iloc[i,:].values, marker='.',s=1 )
        
        prev=None
        for binidx, (chrom, binpos) in enumerate(df_wo_alleles_gc_corr[chrom_order].columns):
            if prev is not None and chrom!=prev:
                ax.axvline(binidx-1,c='k',lw=1, )
            prev = chrom
        ax.set_yticks([0,2,4])
        #ax.set_ylabel('Copy number')
    

In [ ]:
# Sum allele data for all columns: (creates a non-allele specific matrix)

total = df_wo_alleles.sum(1)

df_wo_alleles_pre_normed = (df_wo_alleles[sbins]/bininfo['correct'][sbins])*bininfo['correct'][sbins].mean()

df_wo_alleles_normed = ((df_wo_alleles_pre_normed.T.fillna(0) / df_wo_alleles_pre_normed[sbins].T.mean().fillna(0)).T) * 2

df_wo_alleles_normed=df_wo_alleles_normed[df_wo_alleles_pre_normed.median(1)>0]

df_wo_alleles_normed.loc[:,'total'] = total
fig, ax = plt.subplots(figsize=(10,100))

sns.heatmap(df_wo_alleles_normed.sort_index(1).sort_values('total')['chr9'],ax=ax,vmax=4)
df_wo_alleles_normed=df_wo_alleles_normed.drop('total',1)

In [ ]:
plt.scatter(var,df_wo_alleles.sum(1)[df_wo_alleles_pre_normed.median(1)>0])

In [ ]:
s = df_wo_alleles.sum(1)[df_wo_alleles_pre_normed.median(1)>0].to_frame()
s.columns=['uniquely mapping reads']
v = var.to_frame()
v.columns=['variance of derivative']
s = v.join(s)


s.plot.scatter('uniquely mapping reads','variance of derivative',s=2,c='grey', alpha=0.2)
ax = plt.gca()
ax.set_yscale('log')
ax.set_xscale('log')

ax.scatter(s.loc[segmented_matrix.index]['uniquely mapping reads'],
            s.loc[segmented_matrix.index]['variance of derivative'],
                                           marker='x',s=0.05,c='g')
sns.despine()





In [ ]:
s.loc[segmented_matrix.index].max()

In [ ]:
# Some cells are a bit or very noisy, calculate noise for chromosome 1
var = df_wo_alleles_gc_corr['chr1'].diff(periods=1,axis=1).var(1)
plt.hist( var, bins=100)

var_threshold=0.6

plt.show()
df_wo_alleles_gc_corr.loc[:,'var'] =  var


fig, axes = plt.subplots(10,1,figsize=(10,10), sharex=True, sharey=True)
d = df_wo_alleles_gc_corr[df_wo_alleles_gc_corr['var']<=var_threshold].sort_values('var').drop('var',1)['chr9'].iloc[:10,:]
for i,ax in zip(range(10), axes):
    d.iloc[i,:].plot(style='.',ms=1,ax=ax)


fig, axes = plt.subplots(10,1,figsize=(10,10), sharex=True, sharey=True)
d = df_wo_alleles_gc_corr[df_wo_alleles_gc_corr['var']<=var_threshold].sort_values('var').drop('var',1)['chr9'].iloc[-10:,:]
for i,ax in zip(range(10), axes):
    d.iloc[i,:].plot(style='.',ms=1,ax=ax)


fig, ax = plt.subplots(figsize=(10,100))
sns.heatmap(df_wo_alleles_gc_corr.sort_index(1).sort_values('var')['chr9'],ax=ax,vmax=4)


best_cells  = df_wo_alleles_gc_corr[df_wo_alleles_gc_corr['var']<=var_threshold].index

fig, ax = plt.subplots(figsize=(10,100))
sns.heatmap(df_wo_alleles_gc_corr.loc[best_cells].sort_index(1).sort_values('var')['chr9'],ax=ax,vmax=4)


print(f"{len(best_cells)} cells left")

df_wo_alleles_gc_corr.drop('var',1,inplace=True)

### Perform median 2 normalisation 

In [ ]:
tsne = TSNE()
tsne_X = tsne.fit_transform( df_wo_alleles_gc_corr.loc[best_cells] )
tsne_X_before_select = tsne.fit_transform( np.clip(df_wo_alleles_gc_corr.fillna(0), 0,MAXCP))

In [ ]:
k = sklearn.cluster.KMeans(n_clusters=12)
clusters = k.fit_predict(tsne_X)

In [ ]:
pargs = {'s':8, 'cmap':plt.get_cmap('tab20')}

fig, axes = plt.subplots(1,2)
axes[0].scatter( tsne_X_before_select[:,0], tsne_X_before_select[:,1])
axes[0].set_title('T-SNE of CNV data\n without GC correction')

axes[1].scatter( tsne_X[:,0], tsne_X[:,1],  c=clusters,**pargs)
axes[1].set_title('T-SNE of CNV data \nlow variance filter')



In [ ]:
from scipy.ndimage import gaussian_filter
df_norm_gauss = gaussian_filter(df_wo_alleles_gc_corr, (0.01,5))
df_norm_gauss = pd.DataFrame(df_norm_gauss, index=df_wo_alleles_gc_corr.index, columns=df_wo_alleles_gc_corr.columns)

In [ ]:
for cluster in set(clusters):
    fig, ax = plt.subplots()
    sns.heatmap(df_wo_alleles_gc_corr.loc[best_cells,:][clusters==cluster], vmax=MAXCP )
    #df_norm_gauss= df_norm_gauss.drop('cluster',1)
    plt.title(cluster)

In [ ]:
noise_cluster= 4

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(oob_score=True,random_state=42)
y = clusters==noise_cluster

rf.fit(df_wo_alleles_gc_corr.loc[best_cells],y)
print(rf.oob_score_)

In [ ]:
plt.plot(rf.feature_importances_)

In [ ]:
noise_predictions = rf.predict_proba(df_wo_alleles_gc_corr)[:,0]
df_wo_alleles_gc_corr.loc[:,'p']= noise_predictions
fig, ax = plt.subplots(figsize=(20,60))
#df_wo_alleles_gc_corr['cluster']=clusters
sns.heatmap( df_wo_alleles_gc_corr.sort_values('p')['chr18'],ax=ax, vmax=4)
df_wo_alleles_gc_corr.drop('p',1,inplace=True)

In [ ]:
threshold = 0.99
plt.hist(noise_predictions, bins=100)
plt.axvline(threshold,c='r')
df_wo_alleles_gc_corr[ noise_predictions>=threshold].shape

In [ ]:
fig, ax = plt.subplots(figsize=(20,80))
#df_wo_alleles_gc_corr['cluster']=clusters
selected_cells_for_segmentation = (noise_predictions>=threshold)
sns.heatmap( df_wo_alleles_gc_corr[selected_cells_for_segmentation], vmax=4)

In [ ]:
sns.clustermap( df_wo_alleles_gc_corr[selected_cells_for_segmentation], 
               vmax=5, vmin=0, col_cluster= False, method='ward', cmap='bwr',figsize=(20,60))
plt.savefig('./binned_200k_clustered_other_corr_reads_rf.png',dpi=200)

In [ ]:
import scipy.signal

fig, ax = plt.subplots()
sf = df_wo_alleles_gc_corr.loc[best_cells][ clusters==noise_cluster ]
high_peaks = scipy.signal.find_peaks( sf.sum() )[0]
low_peaks = scipy.signal.find_peaks( -sf.sum() )[0]
sf.sum().plot()
plt.scatter(high_peaks,sf.sum()[high_peaks], c='r')
plt.scatter(low_peaks,sf.sum()[low_peaks], c='g')

In [ ]:
weight = 1

fig, ax = plt.subplots()
#drate = df_norm.iloc[:,low_peaks].mean(1) # (df_norm.iloc[:,low_peaks].sum(1)*weight).div( (df_norm.iloc[:,low_peaks].sum(1)*weight)+df_norm.iloc[:,high_peaks].sum(1))
#drate = (df_norm.iloc[:,low_peaks].sum(1)).div( df_norm.sum(1) )
drate = (df_wo_alleles_gc_corr.iloc[:,low_peaks].replace([np.inf, -np.inf], np.nan).fillna(2).mean(1)).div( 
        (df_wo_alleles_gc_corr.iloc[:,low_peaks].replace([np.inf, -np.inf], np.nan).fillna(2)).mean(1)+
        df_wo_alleles_gc_corr.iloc[:,high_peaks].replace([np.inf, -np.inf], np.nan).fillna(2).mean(1))

#drate = (drate-drate.mean()).abs()
plt.hist( 
    drate,
    bins=150
)


plt.hist((sf.iloc[:,low_peaks].replace([np.inf, -np.inf], np.nan).fillna(2).mean(1)).div( 
        (sf.iloc[:,low_peaks].replace([np.inf, -np.inf], np.nan).fillna(2)).mean(1)+
        sf.iloc[:,high_peaks].replace([np.inf, -np.inf], np.nan).fillna(2).mean(1)),color='r')



In [ ]:
fig, axes = plt.subplots(10,1,figsize=(10,10), sharex=True, sharey=True)
for i,ax in zip(range(10), axes):
    df_wo_alleles_gc_corr[selected_cells_for_segmentation].sort_index(1)[chrom_order].iloc[i,:].plot(style='.',ms=1,ax=ax)

In [ ]:
import logging
log = logging.getLogger()
logging.basicConfig(level=logging.WARN)



def cbs_stat(x):
    '''Given x, Compute the subinterval x[i0:i1] with the maximal segmentation statistic t. 
    Returns t, i0, i1'''
    
    x0 = x - np.mean(x)
    n = len(x0)
    y = np.cumsum(x0)
    e0, e1 = np.argmin(y), np.argmax(y)
    i0, i1 = min(e0, e1), max(e0, e1)
    s0, s1 = y[i0], y[i1]
    return (s1-s0)**2*n/(i1-i0+1)/(n+1-i1+i0), i0, i1+1


def tstat(x, i):
    '''Return the segmentation statistic t testing if i is a (one-sided)  breakpoint in x'''
    n = len(x)
    s0 = np.mean(x[:i])
    s1 = np.mean(x[i:])
    return (n-i)*i/n*(s0-s1)**2

def cbs(x, shuffles=1000, p=.05):
    '''Given x, find the interval x[i0:i1] with maximal segmentation statistic t. Test that statistic against
    given (shuffles) number of random permutations with significance p.  Return True/False, t, i0, i1; True if
    interval is significant, false otherwise.'''

    max_t, max_start, max_end = cbs_stat(x)
    if max_end-max_start == len(x):
        return False, max_t, max_start, max_end
    if max_start < 5:
        max_start = 0
    if len(x)-max_end < 5:
        max_end = len(x)
    thresh_count = 0
    alpha = shuffles*p
    xt = x.copy()
    for i in range(shuffles):
        np.random.shuffle(xt)
        threshold, s0, e0 = cbs_stat(xt)
        if threshold >= max_t:
            thresh_count += 1
        if thresh_count > alpha:
            return False, max_t, max_start, max_end
    return True, max_t, max_start, max_end


def rsegment(x, start, end, L=[], shuffles=1000, p=.05):
    '''Recursively segment the interval x[start:end] returning a list L of pairs (i,j) where each (i,j) is a significant segment.
    '''
    threshold, t, s, e = cbs(x[start:end], shuffles=shuffles, p=p)
    log.info('Proposed partition of {} to {} from {} to {} with t value {} is {}'.format(start, end, start+s, start+e, t, threshold))
    if (not threshold) | (e-s < 5) | (e-s == end-start):
        L.append((start, end))
    else:
        if s > 0:
            rsegment(x, start, start+s, L)
        if e-s > 0:
            rsegment(x, start+s, start+e, L)
        if start+e < end:
            rsegment(x, start+e, end, L)
    return L


def segment(x, shuffles=1000, p=.05):
    '''Segment the array x, using significance test based on shuffles rearrangements and significance level p
    '''
    start = 0
    end = len(x)
    L = []
    rsegment(x, start, end, L, shuffles=shuffles, p=p)
    return L


def validate(x, L, shuffles=1000, p=.01):
    S = [x[0] for x in L]+[len(x)]
    SV = [0]
    left = 0
    for test, s in enumerate(S[1:-1]):
        t = tstat(x[S[left]:S[test+2]], S[test+1]-S[left])
        log.info('Testing validity of {} in interval from {} to {} yields statistic {}'.format(S[test+1], S[left], S[test+2], t))
        threshold = 0
        thresh_count = 0
        site = S[test+1]-S[left]
        xt = x[S[left]:S[test+2]].copy()
        flag = True
        for k in range(shuffles):
            np.random.shuffle(xt)
            threshold = tstat(xt, site)
            if threshold > t:
                thresh_count += 1
            if thresh_count >= p*shuffles:
                flag = False
                log.info('Breakpoint {} rejected'.format(S[test+1]))
                break
        if flag:
            log.info('Breakpoint {} accepted'.format(S[test+1]))
            SV.append(S[test+1])
            left += 1
    SV.append(S[-1])
    return SV


def generate_normal_time_series(num, minl=50, maxl=1000):
    '''Generate a time series with num segments of minimal length minl and maximal length maxl.  Within a segment,
    data is normal with randomly chosen, normally distributed mean between -10 and 10, variance between 0 and 1.
    '''
    data = np.array([], dtype=np.float64)
    partition = np.random.randint(minl, maxl, num)
    for p in partition:
        mean = np.random.randn()*10
        var = np.random.randn()*1
        if var < 0:
            var = var * -1
        tdata = np.random.normal(mean, var, p)
        data = np.concatenate((data, tdata))
    return data


def draw_segmented_data(data, S, title=None, ax=None):
    '''Draw a scatterplot of the data with vertical lines at segment boundaries and horizontal lines at means of 
    the segments. S is a list of segment boundaries.'''
    j=sns.scatterplot(range(len(data)),data,color='black',size=.1,legend=None,ax=ax)
    for x in S:
        j.axvline(x)
    for i in range(1,len(S)):
        j.hlines(np.mean(data[S[i-1]:S[i]]),S[i-1],S[i],color='green')
    j.set_title(title)


In [ ]:
#workers.terminate()

def call_segments(row):
    plot=False
    final_segments = {}
    segment_bounds = collections.defaultdict(set)
    max_iter= 10
    for chromosome in chrom_order:

        data = row[chromosome].sort_index()

        p = 0.05
        shuffles=10000
        L = segment( data.values,p=p,shuffles=shuffles )
        S = validate( data.values, L, p=p,shuffles=shuffles)

        segments = list( (start, end) for start, end in list(more_itertools.windowed(S,2) ))
        #segments.append((segments[-1][1],len(data)-1))
        orig_segs = segments
        prev_len = None

        #print(segments)
        prev_bps = S
        iteration = 0
        while (prev_len is None or prev_len != len(prev_bps)) and len(segments)>1 and iteration<max_iter:

            bps = []
            new_segments=[]
            prev=0
            #print('Starting solution:')
            #print(segments)
            prev_len = len(prev_bps)
            for breakpoint, delta_cn in zip(prev_bps[1:], np.diff( [data.iloc[start:end].median() for start, end in segments] )):
                if abs(delta_cn)>0.6:
                    bps.append(breakpoint)
                    #rint((prev,breakpoint), delta_cn)
                    new_segments.append((prev,breakpoint))
                    prev=breakpoint
            # Add breakpoint to end:
            #new_segments.append((prev,len(data)-1))
            segments = new_segments
            #print('Current solution:')
            if not 0 in bps:
                bps_including_ends = [0]+bps
            else:
                bps_including_ends = bps
            if bps_including_ends[-1] != len(data)-1:
                bps_including_ends+=[len(data)-1]


            called_segment_indices = list(more_itertools.windowed(bps_including_ends,2) )
            segments = called_segment_indices
            #print(segments, bps_including_ends)
            prev_bps = bps_including_ends
            iteration+=1
        
        for bp in bps_including_ends:
            segment_bounds[chromosome].add(bp)
                
        if plot:
            fig,ax = plt.subplots( figsize=(15,2))
            ax.scatter(data.index, data.values,s=1, c='grey')
            for start,end in orig_segs:
                plt.axvline(data.index[start],c='k',ls=':',lw=1)
                plt.axvline(data.index[end-1],c='k',ls=':',lw=1)
            for x in segment_bounds[chromosome]:
                plt.axvline(data.index[x],c='r')
            plt.title(chromosome)
            plt.show()
        final_segments[chromosome] = called_segment_indices
    return final_segments, row.name
workers = Pool(32)

In [ ]:
from scipy.cluster.hierarchy import linkage,fcluster

random.seed(42)

# Roughly cluster the cells to generate groups from which to extract segment calls..
hand_picked_thresholds = {
    'chr1':3,
    'chr2':7,
    'chr3':3,
    'chr4':2,
    'chr5':1,
    'chr7':2,
    'chr8':7,
    'chr9':6,
    'chr10':4,
    'chr11':2,
    'chr12':3,
    'chr18':5,
    'chrX':5    
}

MAXCP=4
segment_bounds = collections.defaultdict(set)
segment_calls = []
for chromosome in chrom_order:
    d = df_wo_alleles_gc_corr[selected_cells_for_segmentation][chromosome].clip(0,MAXCP) 
    L = linkage(d, method='ward')

    scores = []
    thresholds = list(range(1,12))
    cluster_count = []

    max_value = None
    max_threshold = None
    max_clustering = None
    for t in thresholds:
        z = fcluster(L, t ,'maxclust')
        cluster_count.append(len(set(z)))
        try:
            scores.append( sklearn.metrics.silhouette_score(d,z) )
        except Exception as e:
            scores.append(0)
            pass
        
        if chromosome in hand_picked_thresholds:
            if t==hand_picked_thresholds[chromosome]:
                max_value = scores[-1]
                max_clustering = z
                max_threshold = t
                print(t)

        elif max_value is None or scores[-1]>max_value :
            max_value = scores[-1]
            max_clustering = z
            max_threshold = t


    plt.plot(thresholds,scores)
    plt.title(chromosome)
    plt.gca().axvline(max_threshold,c='r')
    plt.show()
    
    assignments = max_clustering
    cdf = pd.DataFrame( [assignments],  columns=d.index )
    cdf, lut = createRowColorDataFrame(cdf.T)

    sns.clustermap( d.sort_index(1),
                   col_cluster= False, row_cluster=True, method= 'ward', vmax=MAXCP, row_colors=cdf, figsize=(20,40))
    plt.show()
    
    
    assignments = max_clustering
    
    delta_cn_hist= collections.Counter()
    fig, axes = plt.subplots(len(set(assignments)),1,figsize=(8,1 + len(set(assignments))), sharex=True, sharey=True, squeeze=False)
    
    for ax_col,clust in zip(axes, sorted(list(set(assignments)))):
        ax = ax_col[0]
        ax.set_title(f'{chromosome}, cluster {clust}')
        data = d[assignments==clust].median().sort_index(0)

        p = 0.005

        sample = data.values
        L = segment(  sample,p=p )

        # Copy number per segment:
        S = validate(sample, L,p=p)
        
        ax.set_ylim(0, MAXCP+0.5)
        #draw_segmented_data(sample,  S, title=f'{chromosome}, cluster {clust}', ax=ax)

        segments = list(more_itertools.windowed(S,2) )
        
        bps = []
        # @todo: this code has a bug
        for breakpoint, delta_cn in zip(S[1:], np.diff( [data.iloc[start:end].median() for start, end in segments] )):
            delta_cn_hist[delta_cn] += 1

            if abs(delta_cn)>0.6:
                #egment_bounds[chromosome][data.index[min(breakpoint,len(data)-1)]]+=len(data)
                bps.append(breakpoint)
        
        
        bps_including_ends = [1]+bps+[len(data)-2]
        
        called_segment_indices = list(more_itertools.windowed(bps_including_ends,2) )
        
        for bp in bps_including_ends:
            ax.axvline(np.array(data.index.values, dtype=int)[bp] * bin_size, c='r')
            segment_bounds[chromosome].add(bp)
            
        for (start, end) in called_segment_indices:
            
            data_in_seg = data[start:end]
            
            color = 'grey'
            if data_in_seg.median()>2.5:
                color='r'
            if data_in_seg.median()<1.5:
                color='b'
            
            ax.scatter( np.array(data_in_seg.index.values, dtype=int)*bin_size, data_in_seg.values, s=4,c=color  )
        
        ax.set_ylabel('copy number')
        sns.despine(ax=ax)
        
        for cp in range(1,MAXCP+1):
            ax.axhline(cp,c='k',lw=0.5)
    
        
    ax.set_xlabel('position')
    plt.tight_layout()
    plt.show()
    plt.savefig(f'./segments/{chromosome}.png',dpi=100)
    plt.close()
        

In [ ]:
c = (df['allele1'].loc[:,sbins]['chr18'].sort_index(1).iloc[:,0:7].sum(1)+
    df['allele1'].loc[:,sbins]['chr18'].sort_index(1).iloc[:,69:76].sum(1))/(
        
    (df['allele2'].loc[:,sbins]['chr18'].sort_index(1).iloc[:,0:7].sum(1)+
    df['allele2'].loc[:,sbins]['chr18'].sort_index(1).iloc[:,69:76].sum(1))+
    (df['allele1'].loc[:,sbins]['chr18'].sort_index(1).iloc[:,0:7].sum(1)+
    df['allele1'].loc[:,sbins]['chr18'].sort_index(1).iloc[:,69:76].sum(1))
)

c = c.loc[df_wo_alleles_gc_corr[selected_cells_for_segmentation].index]

In [ ]:
cid = 2
plt.scatter( 
    
    df_wo_alleles_gc_corr[selected_cells_for_segmentation]['chr9'].iloc[cid,:].index,
    df_wo_alleles_gc_corr[selected_cells_for_segmentation]['chr9'].iloc[cid,:]


)
plt.show()
plt.scatter(
        df_wo_alleles_gc_corr[selected_cells_for_segmentation]['chr9'].iloc[cid,:].index,
        df['allele1'].loc[:,sbins]['chr9'].sort_index(1).loc[df_wo_alleles_gc_corr[selected_cells_for_segmentation].index].iloc[cid]

        )
    
plt.scatter(
        df_wo_alleles_gc_corr[selected_cells_for_segmentation]['chr9'].iloc[cid,:].index,
        -df['allele2'].loc[:,sbins]['chr9'].sort_index(1).loc[df_wo_alleles_gc_corr[selected_cells_for_segmentation].index].iloc[cid]

        )    

In [ ]:
q = collections.Counter( df_wo_alleles_gc_corr[selected_cells_for_segmentation]['chr9'].idxmax(axis=1) )
plt.scatter(list(q.keys()),list(q.values()))

In [ ]:
plt.hist(df_wo_alleles_gc_corr[selected_cells_for_segmentation]['chr6'].mean(1), bins=150)

In [ ]:
plt.hist(df_wo_alleles_gc_corr[selected_cells_for_segmentation]['chr9'].iloc[:,delta:6].mean(1), bins=50)
plt.show()


plt.hist(
    pd.concat((
        df_wo_alleles_gc_corr[selected_cells_for_segmentation]['chr9'].iloc[:,delta:7-delta], 
    df_wo_alleles_gc_corr[selected_cells_for_segmentation]['chr9'].iloc[:,69+delta:76-delta]
    ),axis=1
    ).mean(1), bins=50)

In [ ]:
delta = 0

plt.scatter( 
    df_wo_alleles_gc_corr[selected_cells_for_segmentation]['chr9'].iloc[:,delta:7-delta].mean(1), 
    df_wo_alleles_gc_corr[selected_cells_for_segmentation]['chr9'].iloc[:,69+delta:76-delta].mean(1)
)
plt.show()
plt.scatter( 
    df_wo_alleles_gc_corr[selected_cells_for_segmentation]['chr9'].iloc[:,delta:6-delta].mean(1), 
    pd.concat((
        df_wo_alleles_gc_corr[selected_cells_for_segmentation]['chr9'].iloc[:,delta:7-delta], 
    df_wo_alleles_gc_corr[selected_cells_for_segmentation]['chr9'].iloc[:,69+delta:76-delta]
    ),axis=1
    ).mean(1),
    c=c
    
)
plt.xlim(1,4)
plt.ylim(1,4)
    

In [ ]:
segment_bounds =  {
            'chr1': { 15, 155, 157, 201, 436},
             'chr2': {0, 135, 467},
             'chr3': {0, 252, 387},
             'chr4': {0, 372},
             'chr5': {0, 348},
             'chr6': {0, 55, 66, 68, 333},
             'chr7': {0, 302},
             'chr8': {0, 5, 8, 82, 123, 279},
             'chr9': {0, 7, 8, 69, 76, 216},
             'chr10': {0, 91, 255},
             'chr11': {0, 256},
             'chr12': {0, 142, 181, 258},
             'chr13': {0, 190},
             'chr14': {0, 173},
             'chr15': {0, 10, 16, 153},
             'chr16': {0, 150},
             'chr17': {0, 97, 152},
             'chr18': {0, 29, 32, 37, 42, 147},
             'chr19': {0, 107},
             'chr20': {0, 116},
             'chr21': {0, 63},
             'chr22': {0, 64},
             'chrX': {0, 292}}

In [ ]:
segment_bounds

In [ ]:
final_segments = []
min_segment_size = 5
for chrom, bounds_set in segment_bounds.items():
    bounds_list=sorted(list(bounds_set))
    for seg in list( more_itertools.windowed(bounds_list,2) ):
        if np.abs(np.diff(seg))[0] < min_segment_size:
            continue
        print(chrom, seg, np.abs(np.diff(seg))[0] )
        seg = (seg[0],seg[1]) # the range is exclusive
        final_segments.append( (chrom,seg) )
print(final_segments)

In [ ]:
variances = {}
MAXCP=10 # just to not clip the values here
for chrom, seg in final_segments:
    if not (chrom,seg) in segmented_matrix:
        continue
        
    
    diploid_cells_for_bin = segmented_matrix[chrom,seg]==segmented_matrix[chrom,seg].mode()[0]
    diploid_cells_for_bin=diploid_cells_for_bin[diploid_cells_for_bin].index
    #plt.hist(np.clip(d.loc[diploid_cells_for_bin][chrom].iloc[:,seg[0]:seg[1]],0, MAXCP  ).median(1).values.flatten(),bins=30)
    #plt.title(f'{chrom},{seg}')
    #plt.show()
    variances[chrom,seg] = np.clip(d.loc[diploid_cells_for_bin][chrom].iloc[:,seg[0]:seg[1]],0, MAXCP  ).median(1).values.flatten().var()

vlim = 0.02
vf = pd.DataFrame({'variance':variances})


variance_selected_bins = vf[ vf['variance']<=vlim ].index

vf.plot.bar(figsize=(20,4))
ax = plt.gca()
ax.axhline(vlim,c='red')
plt.show()

vf['variance'].plot.hist()
ax = plt.gca()
ax.axvline(vlim,c='red')


final_segments = []
min_segment_size = 5
for chrom, bounds_set in segment_bounds.items():
    bounds_list=sorted(list(bounds_set))
    for seg in list( more_itertools.windowed(bounds_list,2) ):
        if not (chrom,seg) in variance_selected_bins:
            continue
        print(chrom, seg, np.abs(np.diff(seg))[0] )
        seg = (seg[0],seg[1]) # the range is exclusive
        final_segments.append( (chrom,seg) )
print(final_segments)



In [ ]:
# Segment the copy number matrix:

MAXCP=4

d = df_wo_alleles_gc_corr[selected_cells_for_segmentation]

segmented_matrix_floating=[]
segmented_matrix = []
segmented_matrix_labels = []

min_segment_size = 5

for chrom, seg in final_segments:

    print(chrom,seg)
    segmented_matrix_floating.append(np.clip(d[chrom].iloc[:,seg[0]:seg[1]].median(1),0, MAXCP  ))
    segmented_matrix.append(np.clip(d[chrom].iloc[:,seg[0]:seg[1]].median(1).round(0),0, MAXCP  ))
    segmented_matrix_labels.append((chrom,seg))

segmented_matrix = pd.concat(segmented_matrix,1)
segmented_matrix.columns = segmented_matrix_labels

segmented_matrix_floating = pd.concat(segmented_matrix_floating,1)
segmented_matrix_floating.columns = segmented_matrix_labels

columns_with_info = [len(segmented_matrix[column].unique())>1 for column in segmented_matrix]
segmented_matrix = segmented_matrix.loc[:,columns_with_info]
segmented_matrix_labels = np.array(segmented_matrix_labels)[columns_with_info]
segmented_matrix.columns= pd.MultiIndex.from_tuples(segmented_matrix.columns)

cnv_clusters = collections.Counter()
cell_to_unfiltered_cnv = collections.defaultdict(list)
for cell,row in segmented_matrix.iterrows():
    cnv_clusters[tuple(row)]+=1
    cell_to_unfiltered_cnv[tuple(row)].append(cell)

keep_clusters = []
cell_order = []
cell_cluster_names = []
current_cluster_name = 1
min_cells_per_cluster=2

median_profiles = []

for ci,(cluster, obs) in enumerate( cnv_clusters.most_common() ):
    if obs>=min_cells_per_cluster:
        
        keep_clusters.append(cluster)
        
        cells_in_cluster = []
        for cell in cell_to_unfiltered_cnv[cluster]:
            cells_in_cluster.append(cell)
        
        # Cluster the cells ..
        cells_in_cluster = np.array(cells_in_cluster)
        
        cells_in_cluster = cells_in_cluster[leaves_list( 
                linkage(d.loc[cells_in_cluster], method='ward',optimal_ordering=True)
        )]
        print(obs, len(cells_in_cluster))
        median_profiles.append( d.loc[cells_in_cluster].median(0) )
        
        cell_order += list(cells_in_cluster)
        cell_cluster_names += [current_cluster_name]*len(cells_in_cluster)
        
        current_cluster_name+=1
        fig, ax = plt.subplots()
        sns.heatmap(d.loc[cells_in_cluster].sort_index(1)[chrom_order] ,ax=ax, vmax=MAXCP,cmap='bwr')
        plt.savefig(f'./cluster_{ci}.png')
        plt.close()
    
print(f'{current_cluster_name} clusters identified')
print(f'{segmented_matrix.shape[0]} cells assigned to a cluster')
    


# Correlations between the segments:

In [ ]:
sns.clustermap(segmented_matrix_floating.corr(),yticklabels=True,xticklabels=True,method='single')

In [ ]:
import random
def createRowColorDataFrame( discreteStatesDataFrame, nanColor =(0,0,0), predeterminedColorMapping={},seed=1337 ):

    """ Create color dataframe for use with seaborn clustermap

    Args:
        discreteStatesDataFrame (pd.DataFrame) : Dataframe containing the data to convert to colors, like:  pd.DataFrame( [['A','x'],['A','y']],index=['A','B'], columns=['First', 'Second'] )

        nanColor(tuple) : Color for records having an NAN

        predeterminedColorMapping(dict) : Supply class colors here (optional)

    Returns:
        discreteColorMatrix (pd.DataFrame) : Dataframe to pass to seaborn clustermap row_colors, or col_colors
        
        luts (dict) : class->color mapping
    """
    # Should look like:
    # discreteStatesDataFrame = pd.DataFrame( [['A','x'],['A','y']],index=['A','B'], columns=['First', 'Second'] )
    colorMatrix = []
    luts = {}
    random.seed(seed)
    for column in discreteStatesDataFrame:
        states = [x for x in discreteStatesDataFrame[column].unique() if not pd.isnull(x)]
        undeterminedColorStates = [x for x in discreteStatesDataFrame[column].unique() if not pd.isnull(x) and not x in predeterminedColorMapping]

        cols = sns.color_palette('hls',len(undeterminedColorStates))
        random.shuffle(cols)
        #lut = { i:sns.color_palette('bright').jet(x) for i,x in zip(states, np.linspace(0,1,len(states)) )}
        lut = { state:cols[i] for i,state in enumerate(undeterminedColorStates) }
        lut.update({key:value for key,value in predeterminedColorMapping.items() if key in states})
        lut[np.nan] = nanColor
        colorMatrix.append( [ nanColor if pd.isnull(x) else lut[x] for x in  discreteStatesDataFrame[column] ] )
        luts[column] = lut
    discreteColorMatrix = pd.DataFrame(colorMatrix, index=discreteStatesDataFrame.columns, columns=discreteStatesDataFrame.index ).transpose()
    return discreteColorMatrix, luts

cell_annot_df = pd.DataFrame( [cell_cluster_names, [cell.split('-')[0] for cell in cell_order] ],  columns=cell_order )
cell_annot_df, lut = createRowColorDataFrame(cell_annot_df.T)


bin_annot_df = pd.DataFrame( [[chrom for chrom, pos in d.sort_index(1)[chrom_order].columns] ],  columns=d.sort_index(1)[chrom_order].loc[cell_order].columns )
bin_annot_df, lut = createRowColorDataFrame(bin_annot_df.T)

sns.clustermap(d.sort_index(1)[chrom_order].loc[cell_order], vmax=4,cmap='bwr', row_colors=cell_annot_df, col_colors=bin_annot_df,figsize=(20,30), row_cluster=False, col_cluster=False)
plt.savefig(f'./cluster_sorted.png', dpi=200)

In [ ]:
cell_annot_df['cluster'] = cell_cluster_names

In [ ]:
# Add allele frequency data for chromosome 18 and chromosome 4
baf_18 = df['allele1'].loc[:,sbins]['chr18'].sum(1) / (df['allele1'].loc[:,sbins]['chr18'].sum(1)+df['allele2'].loc[:,sbins]['chr18'].sum(1))
baf_4 = df['allele1'].loc[:,sbins]['chr4'].sum(1) / (df['allele1'].loc[:,sbins]['chr4'].sum(1)+df['allele2'].loc[:,sbins]['chr4'].sum(1))
baf_9 = df['allele1'].loc[:,sbins]['chr9'].sum(1) / (df['allele1'].loc[:,sbins]['chr9'].sum(1)+df['allele2'].loc[:,sbins]['chr9'].sum(1))

In [ ]:
bafs = pd.concat([baf_4, baf_9, baf_18],axis=1)
bafs.columns=['baf_4','baf_9','baf_18']
bafs

In [ ]:
baf_segmented_matrix = []
baf_segmented_matrix_labels = []
baf_min_segment_size = 0

acn_matrix = []

for chrom,seg in segmented_matrix:

    if not chrom in ['chr4']: #, 'chr9', 'chr18']: # @todo
        continue

    if np.abs(np.diff(seg))[0] < min_segment_size:
        continue
    seg_baf = df['allele1'].loc[:,sbins][chrom].iloc[:,seg[0]:seg[1]].sum(1) / (


        df['allele1'].loc[:,sbins][chrom].iloc[:,seg[0]:seg[1]].sum(1)+
        df['allele2'].loc[:,sbins][chrom].iloc[:,seg[0]:seg[1]].sum(1)
    )
    baf_segmented_matrix.append(seg_baf)
    baf_segmented_matrix_labels.append((chrom,seg))

    cn = segmented_matrix[chrom,seg]


baf_segmented_matrix = pd.concat(baf_segmented_matrix,1)

baf_columns_with_info = [len(baf_segmented_matrix[column].unique())>1 for column in baf_segmented_matrix]
baf_segmented_matrix = baf_segmented_matrix.loc[:,baf_columns_with_info]
baf_segmented_matrix_labels = np.array(baf_segmented_matrix_labels)[baf_columns_with_info]
baf_segmented_matrix = pd.DataFrame(baf_segmented_matrix)
baf_segmented_matrix.columns = baf_segmented_matrix_labels

In [ ]:
from matplotlib.ticker import MaxNLocator
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150
# Extract copies per allele 


for col in baf_segmented_matrix.columns:

    
    x = []
    y = []

    
    for cell, row in baf_segmented_matrix.iterrows():
        if not cell in segmented_matrix.index:
            continue
        for k, baf in row.iteritems():
            if k!=col:
                continue
            
            x.append(baf)
            y.append(segmented_matrix_floating.loc[cell][k])

    jitter=0
    fig, axes = plt.subplots(2,1,sharex=True,figsize=(4,6))
    axes[0].hist(y,bins=100)
    axes[0].set_ylabel('# cells in bin' )
    ax = axes[1]
    
    ax.scatter(y,x, s=1,alpha=0.1) #+np.random.random(len(x))*jitter -0.5*jitter , 
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.set_ylim(-0.05,1.05)
    plt.title(f' {col[0]}:{col[1][0]}-{col[1][1]}')
    ax.set_ylabel('B-allele frequency')
    ax.set_xlabel('Estimated copy number')
    ax.set_xlim(-0.5,None)
    plt.show()







In [ ]:
import pandas.core.series
def baf_and_cn_to_allelic_cn(baf, cn):
    """
    Convert b-allele frequency and total copy number to the copy number of both alleles
    
    Args:
        baf(np.array)
        cn(np.array))
    Returns
        (Allele A cn, Allele B cn)
    """
    
    b_allele_cn = np.round(cn*baf,0)
    a_allele_cn = cn - b_allele_cn
    
    for c,allele in zip((a_allele_cn, b_allele_cn),'AB'):
        if type(c) is pandas.core.series.Series:
            c.name = (c.name[0], allele,(c.name[1]) )#f'{c.name[0]}_{n}:{c.name[1][0]}-{c.name[1][1]}'
    
    return a_allele_cn, b_allele_cn
    
baf_and_cn_to_allelic_cn(np.array([0.5,1]),np.array([2,2]))

In [ ]:
allele_cn_matrix = pd.concat(
pd.DataFrame( baf_and_cn_to_allelic_cn(baf,segmented_matrix[seg]) ) 
    for seg,baf in baf_segmented_matrix.loc[segmented_matrix.index].T.iterrows()
    ).T

allele_cn_matrix.columns = pd.MultiIndex.from_tuples(allele_cn_matrix.columns)

In [ ]:
# Add columns without allele information:
non_allele_cn_matrix = segmented_matrix[ 
    [seg for seg in segmented_matrix if not seg in baf_segmented_matrix.columns] ]
non_allele_cn_matrix.columns = pd.MultiIndex.from_tuples([(chrom,'none',(start,end)) for (chrom,(start,end)) in non_allele_cn_matrix.columns])

# Format tuple headers to strings:
hybrid_allele_cn_matrix = non_allele_cn_matrix.join(allele_cn_matrix) 
hybrid_allele_cn_matrix.head()

In [ ]:
MIN_OBS = 1
states = collections.Counter()

for cell, row in hybrid_allele_cn_matrix.iterrows():
    state = tuple(row)
    states[state] += 1

state_ids = {}
cell_to_state = {}
current_state_id = 1
assigned_cells = 0
state_bin_info = []
#'chromosome', 'cluster', 'binIndex', 'startCoordinate', 'endCoordinate'
# _A, _B when allelic
cell_order = []
for state,obs in states.most_common():
    if obs<MIN_OBS:
        continue
        
    for cell, row in hybrid_allele_cn_matrix.iterrows():
        cell_state = tuple(row)
        if cell_state == state:
            cell_order.append(cell)
        
    current_state_id+=1
    assigned_cells+=obs
    
    

In [ ]:
assigned_cells

In [ ]:
# Convert the bin indices to coordinates:
hybrid_allele_cn_matrix.columns = pd.MultiIndex.from_tuples(( (chrom, allele, (start*bin_size, end*bin_size)) 
                           for chrom, allele, (start, end) in hybrid_allele_cn_matrix.columns)
                         )

hybrid_allele_cn_matrix[
    [chrom for chrom in chrom_order if chrom in hybrid_allele_cn_matrix.columns.levels[0] 
    ]].to_pickle('./figures/resources/hybrid_cn_matrix_reduced_segments_variance.pickle.gz')

In [ ]:
hybrid_allele_cn_matrix.shape

In [ ]:
sf = pd.concat(
    (
    df['allele1']['chr4'].loc[cell_order],
    df['allele2']['chr4'].loc[cell_order],
        
    df['allele1']['chr18'].loc[cell_order],
    df['allele2']['chr18'].loc[cell_order],
    d.sort_index(1)[chrom_order].loc[cell_order]
    ), axis=1
)


In [ ]:
hybrid_allele_cn_matrix

In [ ]:
cell_annot_df.to_csv('cell_clusters.csv')
hybrid_allele_cn_matrix.to_csv('hybrid_allele_cn_matrix.csv')

In [ ]:
df

In [ ]:
sns.clustermap(sf, vmax=4,cmap='bwr',figsize=(20,30), row_cluster=False, col_cluster=False)

plt.savefig(f'./cluster_sorted_with_bafs.png', dpi=200)


In [ ]:
baf_segmented_matrix.plot.hist(bins=100)
b_loss = baf_segmented_matrix[baf_segmented_matrix[('chr4', (1, 371))]<0.1].index
a_loss = baf_segmented_matrix[baf_segmented_matrix[('chr4', (1, 371))]>0.9].index


In [ ]:
baf_segmented_matrix.columns

In [ ]:
# Attemp to re-cluster the rejected cells


def extract_median_matrix_segments(d, cnv_clusters, min_segment_size = 5,MAXCP=4)
    
    segmented_matrix_floating=[]
    segmented_matrix = []
    segmented_matrix_labels = []

    min_segment_size = 5

    for chrom, seg in final_segments:

        print(chrom,seg)
        segmented_matrix_floating.append(np.clip(d[chrom].iloc[:,seg[0]:seg[1]].median(1),0, MAXCP  ))
        segmented_matrix.append(np.clip(d[chrom].iloc[:,seg[0]:seg[1]].median(1).round(0),0, MAXCP  ))
        segmented_matrix_labels.append((chrom,seg))

    return 
    
segment_matrix( df_wo_alleles_gc_corr[  noise_predictions<threshold  ] )

    

In [ ]:
baf_segmented_matrix.to_pickle('bafs_segmented.pickle.gz')

In [ ]:
chrom= 'chr18'
sns.heatmap( 
    pd.concat([
    df['allele1'][chrom].loc[cell_annot_df.index].fillna(0) / (
        df['allele1'][chrom].loc[cell_annot_df.index].fillna(0)+
        df['allele2'][chrom].loc[cell_annot_df.index].fillna(0)),
        df_wo_alleles_gc_corr[chrom].loc[cell_annot_df.index]/MAXCP,
    
        
        baf_segmented_matrix.loc[cell_annot_df.index]
        
    ],axis=1))

In [ ]:
#fig, ax = plt.subplots(figsize=(15,80))




sns.clustermap( pd.concat([
    pd.concat([
        df_wo_alleles_gc_corr[chrom].loc[cell_annot_df.index]/MAXCP,
    df['allele1'][chrom].loc[cell_annot_df.index].fillna(0) / (
        df['allele1'][chrom].loc[cell_annot_df.index].fillna(0)+
        df['allele2'][chrom].loc[cell_annot_df.index].fillna(0)),
        
    
        
    ],axis=1).fillna(0.5)
    for chrom in ['chr4','chr18','chr9']

],axis=1) , figsize=(15,80), col_cluster=False, vmax=1 )

In [ ]:
af_mat = []
for cluster in list(set(cell_annot_df['cluster']) ):
    cells = cell_annot_df[cell_annot_df['cluster']==cluster].index

    # Show allele frequencies for single cluster:
    df['allele1']['chr18'].loc[cells].fillna(0).mean().plot()
    (-df['allele2']['chr18'].loc[cells].fillna(0).mean()).plot()
    
    af_mat.append( df['allele1']['chr18'].loc[cells].fillna(0)  )
    
    plt.title(cluster)
    plt.show()
    break

In [ ]:
bd =  pd.DataFrame(
    [d.loc[cell_order]['chr4'].median(1),
     d.loc[cell_order]['chr9'].median(1),
   d.loc[cell_order]['chr18'].median(1)
 ],index=['chr4_median_cn','chr9_median_cn','chr18_median_cn']).T.join(cell_annot_df).join(bafs)
#bd.columns[0] = 'chr4_median'
bd.plot.scatter(y='chr4_median_cn',x='baf_4',c=bd['cluster'],s=4, alpha=0.5)
bd.plot.scatter(y='chr9_median_cn',x='baf_9',c=bd['cluster'],s=4, alpha=0.5)
bd.plot.scatter(y='chr18_median_cn',x='chr9_median_cn',c=bd['cluster'],s=4, alpha=0.5)

bd.plot.scatter(y='chr18_median_cn',x='baf_18',c=bd['cluster'],s=4, alpha=0.5)
bd.plot.scatter(y='baf_4',x='baf_18',c=bd['cluster'],s=4, alpha=0.5)

In [ ]:
segmented_b_allele_matrix = []
segmented_b_allele_matrix_names = []
for contig, (start, end) in segmented_matrix_labels:
    if not contig in ('chr9','chr4','chr18'):
        continue
    print(contig, start, end)
    
    baf = df['allele1'][contig].iloc[:,start:end].sum(1) / (df['allele1'][contig].iloc[:,start:end].sum(1)+df['allele2'][contig].iloc[:,start:end].sum(1))
    segmented_b_allele_matrix.append(baf)
    segmented_b_allele_matrix_names.append(('baf', contig, (start, end)))

In [ ]:
segmented_b_allele_matrix = pd.concat( segmented_b_allele_matrix, axis=1 ).loc[cell_order].fillna(0.5)
segmented_b_allele_matrix.columns = pd.MultiIndex.from_tuples(segmented_b_allele_matrix_names)

In [ ]:
b_allele_red = tsne.fit_transform(segmented_b_allele_matrix.join(segmented_matrix))

In [ ]:
bd =  pd.DataFrame(
    [d.loc[cell_order]['chr4'].median(1),
     d.loc[cell_order]['chr9'].median(1),
   d.loc[cell_order]['chr18'].median(1),

 ],index=['chr4_median_cn','chr9_median_cn','chr18_median_cn']).T.join(cell_annot_df).join(bafs)

bd['tsne_0'] =  b_allele_red[:,0]
bd['tsne_1'] =  b_allele_red[:,1]

#bd.columns[0] = 'chr4_median'
bd.plot.scatter(y='chr4_median_cn',x='baf_4',c=bd['cluster'],s=4, alpha=0.5)
bd.plot.scatter(y='chr9_median_cn',x='baf_9',c=bd['cluster'],s=4, alpha=0.5)
bd.plot.scatter(y='chr18_median_cn',x='chr9_median_cn',c=bd['cluster'],s=4, alpha=0.5)

bd.plot.scatter(y='chr18_median_cn',x='baf_18',c=bd['cluster'],s=4, alpha=0.5)
bd.plot.scatter(y='baf_4',x='baf_18',c=bd['cluster'],s=4, alpha=0.5)
bd.plot.scatter(y='tsne_0',x='tsne_1',c=bd['cluster'],s=4, alpha=0.5)

In [ ]:
merged_bd = bd.join(segmented_b_allele_matrix)

In [ ]:
column

In [ ]:
for column in segmented_b_allele_matrix.columns:
    merged_bd.plot.scatter(y='tsne_0',x='tsne_1',c=merged_bd[column],s=4, alpha=0.5,colormap='viridis',vmax=1,vmin=0)
    plt.title(f'{column[1]}: {column[2][0]}:{column[2][1]}' )

In [ ]:
cmap = plt.get_cmap('viridis')

In [ ]:
for column in segmented_b_allele_matrix.columns:
    cell_annot_df[f'{column[0]}: {column[1][0]}:{column[1][1]}'] = [cmap(c) for c in merged_bd[column]]
    

In [ ]:
sns.clustermap(df_int_dist.sort_index(1)[chrom_order].loc[cell_order], vmax=4,cmap='bwr', row_colors=cell_annot_df, col_colors=bin_annot_df,figsize=(20,30), row_cluster=False, col_cluster=False)
plt.savefig(f'./cluster_sorted_annot_baf.png', dpi=200)

In [ ]:
plt.savefig(f'./cluster_sorted_annot_baf.png', dpi=200)

In [ ]:
# Extract B-allele frequencies per segment

 df['allele1']['chr18'].sum(1) / (df['allele1']['chr18'].sum(1)+df['allele2']['chr18'].sum(1))

In [ ]:
segmented_matrix_labels

In [ ]:
segmented_matrix

In [ ]:
df_int_dist.loc[clean_cells][contig].to_pickle('./clean_cell_data.pickle.gz')

In [ ]:
cnv_matrix = pd.DataFrame( cluster_assignments, columns=df_int_dist.loc[clean_cells].index ).T


In [ ]:
cnv_clusters_dict = {
  cluster_vector:cluster_index  for cluster_index, (cluster_vector, obs) in enumerate(
        collections.Counter( [ tuple(row.values) for i,row in cnv_matrix.iterrows()] ).most_common()
    )
}

In [ ]:

cell_cnv_clusters = [cnv_clusters_dict.get(tuple(row.values)) for i,row in cnv_matrix.iterrows() if tuple(row.values) in cnv_clusters_dict]

In [ ]:
cnv_cluster_assignments = fcluster(linkage(cnv_matrix, method='ward',optimal_ordering=True),33,'maxclust')
cdf = pd.DataFrame( [assignments],  columns=cnv_matrix.index )
cdf, lut = createRowColorDataFrame(cdf.T)

d = sns.clustermap( cnv_matrix,
               col_cluster= False, row_cluster=True, method= 'ward', row_colors=cdf)

In [ ]:
cnv_matrix['cluster'] = cnv_cluster_assignments
cm = sns.clustermap( cnv_matrix.sort_values('cluster').drop('cluster',1), col_cluster= False, row_cluster=False)

In [ ]:
fig, ax = plt.subplots()
sns.heatmap( df_clean.loc[cnv_matrix.sort_values('cluster').drop('cluster',1).index][chrom_order], vmax=4 )

In [ ]:
fig, ax = plt.subplots()
df_int_dist[assignments==1][chrom_order].median().plot(style='.', ms=1)

In [ ]:
# determine breakpoints

for clust in set(assignments):
    fig, ax = plt.subplots()
    df_int_dist.loc[clean_cells][assignments==clust].mean().plot()
    break

In [ ]:
#plt.df_clean[assignments==1]
fig, ax = plt.subplots()
sns.heatmap( df_clean.iloc[assignments.argsort()][chrom_order], vmax=4 )

In [ ]:
fig, ax = plt.subplots()
for 
ax.hist( s[chrom_order].iloc[:,15].values, bins=250 )

In [ ]:
def hist_1d(a):
    return np.histogram(a, bins=50, range=(0,5), density=True)[0]

counts = np.apply_along_axis(hist_1d, axis=1, arr=s[chrom_order].values.T)


In [ ]:
sns.clustermap(counts, col_cluster= False)

In [ ]:
sns.clustermap( s[chrom_order].clip(0,4), col_cluster= False, row_cluster=False, vmax=4, method='ward', cmap='viridis')
plt.savefig('./cleaned_cells_orig_200k.png', dpi=600)



In [ ]:
df_clean.median(1)

In [ ]:
np.argsort( [(0,1),(2,0),],axis=1)

In [ ]:
sns.clustermap( ( (df_clean[chrom_order].T/df_clean[chrom_order].median(1))*2).T, col_cluster= False, vmax=5)
plt.savefig('./cleaned_cells_reassigned.png', dpi=600)

In [ ]:
# Perform clustering per chromosome:

In [ ]:

fig, ax = plt.subplots()
plt.hist( (df_norm_gauss.round().mean(1)), bins=100 )
pass

In [ ]:
# Cells sorted by variance
fig, ax = plt.subplots()
sns.heatmap( 
    ( df_norm_gauss.round()-df_norm_gauss).loc[((( df_norm_gauss.round()-df_norm_gauss).var(1))).sort_values(ascending=False).index]
    , ax=ax, vmax=0.5, cmap='viridis' )
plt.savefig('./residuals.png',dpi=600)

In [ ]:
from scipy.ndimage import gaussian_filter
#gaussian_filter(df_norm, (10,0.01))

fig, ax = plt.subplots()
sns.heatmap( 
    df_norm.loc[df_norm_gauss.round().mean(1).sort_values(ascending=False).index]
    , ax=ax, vmax=3 )
#sns.heatmap( gaussian_filter(df_norm, (10,0.01)).loc[df_norm.var(1).sort_values(ascending=False)[20:200].index], ax=ax, cmap='viridis',vmax=3)

In [ ]:
fig, ax = plt.subplots()
df_norm_gauss.loc[ df_norm_gauss.round().mean(1).sort_values(ascending=False).index[:100]].plot()

In [ ]:
df_norm_gauss.loc[ df_norm_gauss.round().mean(1).sort_values(ascending=False).index[:10]].T.plot(alpha=0.3,lw=1)

In [ ]:
plt.savefig('./variance_cells.png',dpi=600)

In [ ]:
#fig, ax = plt.subplots()
sns.clustermap( np.clip(df_norm,0,4) , cmap= 'viridis', row_cluster=True, col_cluster= False)
plt.savefig('./all_cells.png',dpi=400)

In [ ]:
plt.savefig('./all_cells.png',dpi=600)

In [ ]:
# Allele information is available for:
set( [c for c,i in df['allele1'].sum()[df['allele1'].sum()>0].index] )

In [ ]:
use_allele_info = ['chr18', 'chr4']

In [ ]:
rebin_size = 8

new_df_columns = []
new_df_column_names = []

for contig in use_allele_info:
    for allele in ['allele1','allele2']:
        for i,bins in enumerate( list( more_itertools.chunked( df[allele][contig].columns, rebin_size) ) ):
            if len(bins)!=rebin_size:
                continue
            new_df_columns.append( df[allele][contig][bins].sum(1) )
            new_df_column_names.append( (allele,contig, i))


In [ ]:
allelic_df = pd.DataFrame( new_df_columns, index=pd.MultiIndex.from_tuples(new_df_column_names)).T
allelic_df = (allelic_df.T / df_wo_alleles.median(1)).T
sns.clustermap( allelic_df,vmax=1, col_cluster= False)

In [ ]:
#fig, ax = plt.subplots()
allelic_df = pd.DataFrame( new_df_columns, index=pd.MultiIndex.from_tuples(new_df_column_names)).T
allelic_df = (allelic_df.T / df['unk'].median(1)).T
sns.clustermap( allelic_df, vmax=1, col_cluster= False)

In [ ]:
# Remove unk column from bins with allelic information
df_norm = df_norm.drop([('unk',c) for c in use_allele_info],1)

In [ ]:
[ b for b in more_itertools.chunked( df['allele1'][contig].columns, 10) ]

In [ ]:
df['allele1'] for contig in use_allele_info

In [ ]:
# Remove allele collumns form bins  without allelic information:
df_norm = df_norm[ [(allele, contig, b) for allele, contig, b in df_norm.columns if (allele=='unk' or contig in use_allele_info)] ]

In [ ]:
#fig, ax = plt.subplots()
sns.clustermap( df_norm[[column for column in df_norm if column[1] in ('chr4', 'chr18')] ], vmax=0.2 , col_cluster=False)

In [ ]:
df_norm

In [ ]:
selected_regions = df.columns #['chr1', 'chr4', 'chr18', 'chr9']

In [ ]:
from sklearn.decomposition import PCA, FastICA
from sklearn.manifold import TSNE
import numpy as np
pca = PCA(n_components=2)
pca.fit( np.clip(0,10, df_norm.fillna(0).T))

#ica = FastICA(n_components=10)
#ica_X = ica.fit_transform( np.clip(0,10, df_norm.fillna(0)[selected_regions]))

tsne = TSNE()
tsne_X = tsne.fit_transform( np.clip(0,10, df_clean))


In [ ]:
fig, ax = plt.subplots()
ax.scatter( df.sum(1,level=[1,2])['chr4'].sum(1), df.sum(1,level=[1,2])['chr18'].sum(1)) 

In [ ]:
import matplotlib
baf4 = df['allele1']['chr4'].sum(1) / (df['allele1']+df['allele2'])['chr4'].sum(1)
baf18 = df['allele1']['chr18'].sum(1) / (df['allele1']+df['allele2'])['chr18'].sum(1)

fig, ax = plt.subplots()
s = ax.scatter(
    df['allele1']['chr18'].sum(1) / (df['allele1']+df['allele2'])['chr18'].sum(1), 
    df['allele1']['chr4'].sum(1) / (df['allele1']+df['allele2'])['chr4'].sum(1),
    c= df.sum(1), norm=matplotlib.colors.LogNorm(),
    s=4
)
ax.set_xlabel('chr18 B-allele frequency')
ax.set_ylabel('chr4 B-allele frequency')
plt.colorbar(s)

In [ ]:
import matplotlib.colors

pargs = {'s':8}

fig, axes = plt.subplots(1,3, sharex=True, sharey=True)

axes[0].scatter( tsne_X[:,0], tsne_X[:,1], c= df.sum(1), norm=matplotlib.colors.LogNorm(),**pargs)


axes[1].scatter( tsne_X[:,0], tsne_X[:,1], c= baf4,**pargs)



axes[2].scatter( tsne_X[:,0], tsne_X[:,1], c= baf18,  **pargs)


In [ ]:
pargs

In [ ]:
fig, ax = plt.subplots()
ax.scatter( tsne_X[:,0], tsne_X[:,1],# c= np.clip(0, np.percentile(df.sum(1),98), df.sum(1) ))

In [ ]:
fig, ax = plt.subplots()
ax.scatter( tsne_X[:,0], tsne_X[:,1], #c= np.clip(0,np.percentile(df[selected_regions].var(1), 90) ,df[selected_regions].var(1) ))

In [ ]:
df_norm.iloc[block:block+block_size,:].mean()

In [ ]:
fig, ax = plt.subplots()

block_size = 250
for block in range(0, df_norm.shape[0]-block_size, block_size):
    df_norm.iloc[block:block+block_size,:].mean().plot.hist(ax=ax,bins=np.linspace(0,1,0.1))
    #df_norm.iloc[30:,:]['chr6'].mean().plot()

In [ ]:
df_norm = (df.T.fillna(0) / df[ ['chr2','chr3'] ].T.median()).T
error = np.abs( np.round(df_norm) - df_norm ).sum(1)
#df_norm['ica'] =  tsne_X[:,1] #df_norm.var(1)
df_norm['error'] =  error
df_norm = df_norm.sort_values('error').drop('error',1)
#df_norm = df_norm.sort_values('ica').drop('ica',1)

sns.clustermap( df_norm[selected_regions], vmax=3,row_cluster=False, col_cluster= False )

In [ ]:
df_norm